Introduction & Exchange Rates

In [ ]:
import math
import os
import pandas as pd
import numpy as np
from scipy.optimize import newton

from dotenv import load_dotenv, find_dotenv


In [ ]:
df = pd.read_excel('data/xirr.xlsx', engine='openpyxl')
df

In [ ]:
df['total'] = df.income + df.expenses
df

In [ ]:
def xnpv(rate, values, dates):
    min_date = min(dates)

    return sum([
        value / (1 + rate) ** ((date - min_date).days / 365) for value, date in zip(values, dates)
    ])

def xirr(values, dates):
    return newton(lambda r: xnpv(r, values, dates), 0)

print(xirr(df.total.dropna(), df.date.dropna()))

In [ ]:
load_dotenv(find_dotenv())
API_KEY = os.environ.get('OPX_KEY')

print(API_KEY)

In [ ]:
import requests

API_KEY = os.environ.get('OPX_KEY')

r = requests.get(
    'https://openexchangerates.org/api/latest.json',
    params = {
        'app_id': API_KEY,
        'symbols': 'CAD,USD',
        'show_alternative': 'true'
    }
)
rates_ = r.json()['rates']
rates_

In [ ]:
symbol_from = 'CAD'
symbol_to = 'USD'
value = 3000

value * 1/rates_.get(symbol_to) * rates_.get(symbol_from)


In [ ]:
class CurrencyConverter:
    def __init__(self, symbols, API_KEY):
        self.API_KEY = API_KEY
        self.symbols = symbols
        self._symbols = ','.join([str(s) for s in symbols])

        r = requests.get(
            'https://openexchangerates.org/api/latest.json',
            params = {
                'app_id': self.API_KEY,
                'symbols': self._symbols,
                'show_alternative': 'true'
            }
        )

        self.rates_ = r.json()['rates']
        self.rates_['USD'] = 1

    def convert(self, value, symbol_from, symbol_to, round_output = True):
        try:
            x = (value
                 * 1/self.rates_.get(symbol_from)
                 * self.rates_.get(symbol_to))
            if round_output:
                return round(x,2)
            else:
                return x
        except TypeError:
            print('Unavailable or Invalid Symbol')
            return None

In [ ]:
c = CurrencyConverter(['CAD', 'USD', 'DOGE', 'ETH', 'BTC'], API_KEY)

print(c.convert(3000, 'CAD', 'DOGE'))
print(c.convert(5000, 'USD', 'CAD'))

c.rates_

In [ ]:
df = pd.read_excel('data/xirr.xlsx', engine='openpyxl', sheet_name="irregular")
df['total'] = df.income + df.expenses
df['total'].apply(lambda x: c.convert(x, 'CAD', 'DOGE'))

df.dropna()

Debt & Amoritization

In [ ]:
import numpy as np


In [ ]:
loan = 3000
rate = 0.0575
term = 14
balance = loan

df = pd.DataFrame({
    'month': [0],
    'payment': [np.NaN],
    'interest': [np.NaN],
    'principal': [np.NaN],
    'balance': [balance]
})

payment = np.round(-np.pmt(rate/12, term, loan), 2)
print(payment)

for i in range(1, term + 1):
    interest = round(rate/12 * balance, 2)
    principal = payment - interest
    balance = balance - principal
    df = df.append(
        pd.DataFrame({
            'month': [i],
            'payment': [payment],
            'interest': [interest],
            'principal': [principal],
            'balance': [balance]
        })
    )

df = df.reset_index(drop=True)
df[['month', 'payment', 'interest', 'principal', 'balance']]

In [ ]:
loan = 3000
rate = 0.0575
term = 14
balance = loan
payment = np.round(-np.pmt(rate/12, term, loan), 2)

index = range(0, term+1)
columns = ['month', 'payment', 'interest', 'principal', 'balance']
df = pd.DataFrame(index=index, columns=columns)

df.iloc[0]['month']=0
df.iloc[0]['balance']=balance

for i in range(1,term+1):
    interest = round(rate/12 * balance, 2)
    principal = payment - interest
    balance = balance - principal

    df.iloc[i]['month']=i
    df.iloc[i]['payment']=payment
    df.iloc[i]['interest']=interest
    df.iloc[i]['principal']=principal
    df.iloc[i]['balance']=balance
df

In [ ]:
def am(loan, rate, term):
    payment = np.round(-np.pmt(rate/12, term, loan), 2)
    balance = loan
    index = range(0, term+1)
    columns = ['month', 'payment', 'interest', 'principal', 'balance']
    df = pd.DataFrame(index=index, columns=columns)

    df.iloc[0]['month']=0
    df.iloc[0]['balance']=balance

    for i in range(1,term+1):
        interest = round(rate/12 * balance, 2)
        principal = payment - interest
        balance = balance - principal

        df.iloc[i]['month']=i
        df.iloc[i]['payment']=payment
        df.iloc[i]['interest']=interest
        df.iloc[i]['principal']=principal
        df.iloc[i]['balance']=balance

    return df

In [ ]:
loan = 3000

pineapple = am(loan,0.0575, 14)
orange = am(loan,0.0399,20)
banana = am(loan, 0.0889,8)

print(banana['interest'].sum())
print(orange['interest'].sum())
print(pineapple['interest'].sum())

In [ ]:
import datetime

date_1 = datetime.datetime.now()
print(date_1)
print(date_1.__repr__())
print(type(date_1))

date_2 = datetime.datetime.today()
print(date_2)
print(date_2.__repr__())
print(type(date_2))

In [ ]:
import pandas as pd
#print(pd.Timestamp(date_1).normalize())
#print(pd.Timestamp(date_1))

TODAY = pd.Timestamp('today').normalize()
END = TODAY + datetime.timedelta(days=365)

calendar = pd.DataFrame(index=pd.date_range(start=TODAY, end=END))

#semi-monthly
sm = pd.date_range(start=TODAY, end=END, freq='SM')

#monthly
ms = pd.date_range(start=TODAY, end=END, freq='MS')

#build cash flows
income = pd.DataFrame(
    data={'income': 1000},
    index=pd.date_range(start=TODAY, end=END, freq='SM')
)

rent = pd.DataFrame(
    data={'rent': -1500},
    index=pd.date_range(start=TODAY, end=END, freq='MS')
)
#add a starting balance
bank = pd.DataFrame(
    data={'bank': 2000},
    index=pd.date_range(start=TODAY, end=TODAY)
)

#stich calendar together (only works with normalized time stamps)
calendar = pd.concat([calendar, income], axis=1).fillna(0)
calendar = pd.concat([calendar, rent], axis=1).fillna(0)
calendar = pd.concat([calendar,bank], axis=1).fillna(0)

#check specific parts of the calendar
def index_calendar(start, end):
    return calendar.loc[
        (calendar.index >= start) &
        (calendar.index <= end)
]

#print(index_calendar('2021-1-30', '2021-2-5'))

#totals
calendar['total'] = calendar.sum(axis=1)
calendar['cum_total']=calendar['total'].cumsum()
calendar.head(10)


In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.figure(figsize=(10,5))
plt.plot(calendar.index, calendar.total, label='Daily Total')
plt.plot(calendar.index, calendar.cum_total, label='Cumulative Total')
plt.legend()

In [ ]:
def update_totals(df):
    if df.columns.isin(['totals', 'cum_total']).any():
        df['total']=0
        df['cum_total']=0
    df['total']=df.sum(axis=1)
    df['cum_total'] = df['total'].cumsum()
    return df

calendar = update_totals(calendar)
calendar.tail(1)

In [ ]:
def plot_budget(df):
    plt.figure(figsize=(10,5))
    plt.plot(calendar.index, calendar.total, label='Daily Total')
    plt.plot(calendar.index, calendar.cum_total, label='Cumulative Total')
    plt.legend()

plot_budget(calendar)


In [ ]:
vacation = pd.DataFrame(
    data={'vacation': -2500},
    index=[pd.Timestamp('2021-07-01').normalize()]
)

calendar = pd.concat([calendar,vacation], axis=1).fillna(0)
calendar = update_totals(calendar)
plot_budget(calendar)

calendar = calendar.drop('vacation', axis=1)

In [ ]:
import yaml

budget = yaml.load('''
bank:
    frequency: today
    amount: 2000
income:
    frequency: every 2 weeks on Friday
    amount: 1000
rent:
    frequency: every month
    amount: -1500
fun:
    frequency: every week on Friday and Saturday
    amount: -40
''')

budget

calendar = pd.DataFrame(index=pd.date_range(start=TODAY, end=END))

for k,v in budget.items():
    frequency=v.get('frequency')
    amount = v.get('amount')
    dates = get_dates(frequency)
    i = pd.DataFrame(
        data ={k:amount},
        index=pd.DatetimeIndex(pd.Series(dates))
    )
    calendar = pd.concat([calendar, i], axis=1).fillna(0)

calendar['total']=calendar.sum(axis=1)
calendar['cum_total']=calendar['total'].cumsum()
plot_budget(calendar)

In [ ]:
targets = {
    'AMZN': 0.4,
    'CSCO': 0.3,
    'GE': 0.3
}

import pandas as pd
import numpy as np

portfolio = pd.DataFrame(
    index=list(targets.keys())+ ['CASH'],
    data={
        'date': '2018-01-01',
        'price': [np.NaN, np.NaN, np.NaN, 1],
        'target': [0.4,0.3,0.3,0],
        'allocation':[0,0,0,1],
        'shares': [0,0,0,10000],
        'market_value':[0,0,0,10000]
    }
)
portfolio

In [ ]:
def instantiate_portfolio(targets, starting_balance):
    targets['CASH'] = 0
    tickers = list(targets.keys())
    
    df = pd.DataFrame(
        index=tickers,
        columns=[
            'date', 'price', 'target','allocation','shares', 'market_value'
        ]
    )
    df.shares = 0
    df.market_value = 0
    df.allocation = 0
    df.update(
        pd.DataFrame
            .from_dict(targets, orient='index')
            .rename(columns={0:'target'})
    )
    df.at['CASH', 'shares'] = starting_balance
    
    return df

In [ ]:
portfolio = instantiate_portfolio({
    'AMZN': 0.4, 'CSCO': 0.3, 'GE': 0.3},
    10000
)

portfolio

In [ ]:
def update_prices(portfolio, prices):
    prices['CASH'] = 1
    portfolio.update(pd.DataFrame({'price':prices}))
    portfolio.date = prices.name
    portfolio.market_value = portfolio.shares * portfolio.price
    
prices = pd.Series(
        name = '2020-01-01',
        data = {'AMZN':945.21, 'CSCO':30.52, 'GE': 29.27}
)

update_prices(portfolio, prices)
print(portfolio)

In [ ]:
def get_order(portfolio):
    total_value = portfolio.market_value.sum()
    
    order = (
        (total_value * portfolio.target // portfolio.price) - portfolio.shares).drop('CASH')
    return order

In [ ]:
order = get_order(portfolio)
order

In [ ]:
def deposit(portfolio, amount):
    portfolio.at['CASH', 'shares'] += amount
    portfolio.at['CASH', 'market_value'] = portfolio.at['CASH', 'shares']

deposit(portfolio, 1000)

In [ ]:
def simulate_process_order(portfolio, order):
    starting_cash = portfolio.at['CASH', 'shares']
    cash_adjustment = np.sum(order*portfolio.price)
    portfolio.shares += order
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.at['CASH', 'shares'] = starting_cash - cash_adjustment
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.allocation = (
        portfolio.market_value / portfolio.market_value.sum()
    )

simulate_process_order(portfolio, order)
portfolio

In [ ]:
import requests
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

API_KEY = os.environ.get('AV_KEY')
TODAY = pd.Timestamp.today().normalize()

In [ ]:
API_KEY

In [ ]:
def get_price(ticker, outputsize = 'compact', most_recent=False):
    URL = 'https://www.alphavantage.co/query?'
    payload = {
        'function':'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': ticker,
        'apikey': API_KEY,
        'outputsize':outputsize
    }
    r = requests.get(URL, params=payload)
    p = pd.DataFrame(r.json()['Time Series (Daily)']).T['4. close']
    #q = pd.DataFrame(r.json()['Time Series (Daily)'])
    #print(q)
    df = pd.DataFrame({ticker: p.apply(float)})
    df.index = pd.to_datetime(df.index)
    if most_recent:
        return df.tail(1)
    return df

print(get_price('AMZN')[:10])

In [ ]:
def get_historical(tickers, start_date, end_date):
    df = pd.DataFrame(index=pd.date_range(start_date, end_date, freq='D'))
    for t in tickers:
        df = pd.concat([
            df,
            get_price(t, outputsize='full')],
            axis=1,
        )
        df = df.reindex(df.index)
    df = df.fillna(method='ffill').dropna()

    return df
    

In [ ]:
historical_prices = get_historical(
    tickers = ['AMZN', 'CSCO', 'GE'],
    start_date = pd.Timestamp(2016, 1, 1),
    end_date = TODAY
)
print(historical_prices.tail())

In [ ]:
prices = historical_prices.loc['2016-01-04']
prices

In [ ]:
portfolio = instantiate_portfolio(targets, 10000.00)
prices = historical_prices.loc['2016-01-04']
update_prices(portfolio, prices)
order = get_order(portfolio)
simulate_process_order(portfolio, order)
portfolio.market_value.sum()

print(portfolio)

In [ ]:
dates = pd.date_range('2017-01-01', '2017-12-31', freq='Q').tolist()
for d in dates:
    prices = historical_prices.loc[d]
    update_prices(portfolio, prices)
    order = get_order(portfolio)
    print(f'{d}:\n{order}')
    simulate_process_order(portfolio, order)

portfolio.market_value.sum()